In [1]:
%load_ext autoreload
%autoreload 2

In [ ]:


from config import filenames, folders

from matplotlib import pyplot as plt

device = 'cpu'

config = {
    'model_name': 'LSTM_3',
    'feature': 'max-regularization-fine-tune',
    'max_len': 42,
    'min_freq_src': 5,
    'min_freq_trg': 5,

    'embedding_dim': 120,
    'hidden_size': 360,
    'num_layers': 3,

    'num_epochs': 15,
    'weight_decay': 2e-5,
    'label_smoothing': 0.1,

    'dropout_emb': 0.25,

    'dropout_enc': 0.5,
    'dropout_dec': 0.5,

    'dropout_attention': 0.15,

    'learning_rate': 3e-4,
    'gamma': 0.2,
    'patience': 1,
    'threshold': 2e-4,
    'batch_size': 128,

    'use_tf': False,
    'tf_from_epoch': 0,
    'tf_start': 0.9,
    'tf_decrease': 0.02
}

config['weights'] = '../weights/saves/lstm-save-15.pt'

def plot_losses(train_losses, val_losses):
    plt.figure(figsize=(10, 6))
    plt.plot(train_losses, label="Training Loss")
    plt.plot(val_losses, label="Validation Loss")
    plt.xlabel("Epoch")
    plt.ylabel("Loss")
    plt.title("Training and Validation Loss Over Epochs")
    plt.legend()
    plt.grid()
    plt.show()

In [4]:
from dataset import Vocab
vocab_src = Vocab(filenames['train_src'], min_freq=config['min_freq_src'])
vocab_trg = Vocab(filenames['train_trg'], min_freq=config['min_freq_trg'])

In [5]:
print(len(vocab_src))
print(len(vocab_trg))
config['src_vocab_size'] = len(vocab_src)
config['trg_vocab_size'] = len(vocab_trg)

24991
18710


In [6]:
from dataset import TranslationDataset
train_dataset = TranslationDataset(vocab_src, 
                                vocab_trg, 
                                filenames['train_src'], 
                                filenames['train_trg'], 
                                max_len=config['max_len'], 
                                device=device)
val_dataset = TranslationDataset(vocab_src, 
                                vocab_trg, 
                                filenames['test_src'], 
                                filenames['test_trg'], 
                                max_len=72, 
                                device=device, 
                                sort_lengths=False)


100%|██████████| 986/986 [00:00<00:00, 16208.89it/s]


In [8]:
from lstm3 import train, LSTM_3
model = LSTM_3(config=config)

In [ ]:
from submission import get_bleu
from dataset import TestDataLoader, RawDataset

val_loader = TestDataLoader(val_dataset, batch_size=1)

raw = RawDataset(filenames['test_src'])

# get_bleu(model, val_loader, vocab_trg, filenames, device=device, raw_dataset=raw)

In [33]:
for src, trg in val_loader:
    model.inference_beam(src, verbose=True, max_len=12, vocab_trg=vocab_trg, vocab_src=vocab_src, device=device)
    break
print('over')

k: 5	B: 2	max_len: 12
maxlen:12

step:0
beam tokens:
<BOS> als ich <NUM> jahre alt war , wurde ich eines morgens von den klängen heller freude geweckt . <EOS>
index in batch: 0
beam probabbilites:
100.0 0.0 0.0 0.0 0.0
<BOS>
<BOS>
<BOS>
<BOS>
<BOS>

<BOS> mein vater hörte sich auf seinem kleinen , grauen radio die <UNK> der bbc an . <EOS> <PAD> <PAD>
index in batch: 1
beam probabbilites:
100.0 0.0 0.0 0.0 0.0
<BOS>
<BOS>
<BOS>
<BOS>
<BOS>


step:1
beam tokens:
<BOS> als ich <NUM> jahre alt war , wurde ich eines morgens von den klängen heller freude geweckt . <EOS>
index in batch: 0
beam probabbilites:
67.0 6.1 2.5 2.5 2.2
<BOS> when
<BOS> and
<BOS> now
<BOS> so
<BOS> by

<BOS> mein vater hörte sich auf seinem kleinen , grauen radio die <UNK> der bbc an . <EOS> <PAD> <PAD>
index in batch: 1
beam probabbilites:
67.0 4.5 3.7 1.7 1.7
<BOS> my
<BOS> and
<BOS> so
<BOS> now
<BOS> dad


step:2
beam tokens:
<BOS> als ich <NUM> jahre alt war , wurde ich eines morgens von den klängen heller freud